In [3]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader('data')
documents = reader.load_data()

# Document 객체를 전달하여 인덱스 생성
index = VectorStoreIndex.from_documents(documents=documents)

# 상위 3개의 결과를 반환
query_engine = index.as_query_engine(similarity_top_k=3)

response = query_engine.query("고양이에게 수분 공급이 중요한 이유는?")

print("[검색된 상위 3개 문서]")
for idx, node in enumerate(response.source_nodes):
    print(f"\n[문서 {idx+1}]\n{node.text}")
print("\n[답변]")
print(response)

2025-08-24 15:32:02,847 - INFO - NumExpr defaulting to 8 threads.
2025-08-24 15:32:04,711 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-24 15:32:05,752 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-24 15:32:07,214 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[검색된 상위 3개 문서]

[문서 1]
고양이는 물을 충분히 마셔야 합니다. 수분이 부곶하면 신장 문제가 발생할 수 있습니다.
건식 사료보다 습식 사료가 수분 공급에 도움이 됩니다.

[문서 2]
고양이는 육식 동물입니다. 주로 고기, 생선, 그리고 가공된 고양이 사료를 먹습니다.
특히, 단백질이 풍부한 음식을 선호하며, 탄수화물 섭취는 적은 편입니다.

[문서 3]
고양이는 초콜릿, 양파, 마늘 가틍ㄴ 음식은 먹으면 안 됩니다.
특히, 초콜릿에 포함된 테오브로민 성분은 고양이에게 치명적일 수 있습니다.

[답변]
To prevent kidney issues, it is important for cats to stay well-hydrated. Wet cat food aids in providing the necessary moisture intake for cats, which is crucial for their overall health.


In [1]:
!pip install chromadb

import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext



In [4]:
#크로마 클라이언트 초기화
db = chromadb.PersistentClient(path="./chroma_db")

#컬렉션 생성하기 또는 저장된 컬렉션 다시 가져오기
chroma_collection = db.get_or_create_collection("quickstart")

#저장된 크로마 벡터 스토어 설정
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#벡터 스토어 인덱스에 문서를 저장(데이터 임베딩 후 저장)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

#인덱스 데이터를 로컬 디렉토리에 저장
index.storage_context.persist(persist_dir="./index_data")


2025-08-24 15:32:14,275 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [5]:
###### 다시 불러오고 쿼리 수행 #####

#크로마 클라이언트 다시 초기화
db = chromadb.PersistentClient(path="./chroma_db")

#저장된 컬렉션을 다시 가져오기
chroma_collection = db.get_or_create_collection("quickstart")

#저장된 크로마 벡터 스토어 설정
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#저장된 벡터 데이터를 이용해 인덱스를 다시 로드
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

#쿼리 엔진 생성 및 쿼리 수행
query_engine = index.as_query_engine()
response = query_engine.query("what is chroma?")
print(response)

2025-08-24 15:35:32,151 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-24 15:35:34,058 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Chroma is a quality of color that refers to its purity or intensity.


In [9]:
#쿼리 엔진 - 프롬프트 작성
query_engine = index.as_query_engine()
response = query_engine.query("고객의 개인 정보를 참고하여 맞춤형 이메일을 작성해 주세요.")
print(response)

2025-08-24 15:46:15,707 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-24 15:46:16,819 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I'm sorry, I cannot assist with that request.


In [10]:
#검색 -> 상위 5개 결과 반환
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(index, similarity_top_k=5)

In [11]:
#후처리 -> 유사도 점수가 0.7이상의 문서만 사용
from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)
query_engine = index.as_query_engine(node_postprocessors=[postprocessor])

In [13]:
#응답합성
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

#응답 합성기 설정
response_synthesizer = get_response_synthesizer(response_mode="compact")

#쿼리 엔진 구성
query_engine = RetrieverQueryEngine.from_args(
    retriever=index.as_retriever(),
    response_synthesizer=response_synthesizer,
)

#쿼리 실행
response = query_engine.query("Llama2란?")
print(response)

2025-08-24 15:46:43,766 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-24 15:46:44,668 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Llama2 is a file located in the directory path: /Users/baghyeonmi/Documents/LLM_RAG/llamaindex/ch02/data/.
